In [12]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
#data import
path = "mnist_all/"
train = []
test = []
for i in range(10):
    train.append(np.loadtxt(path+f"train{i}.txt"))
    test.append(np.loadtxt(path+f"test{i}.txt"))

y_train = []
y_test = []
for i in range(len(train)):
    yi_train = np.zeros(len(train[i]), dtype=int)+i
    y_train.append(yi_train)

    yi_test = np.zeros(len(test[i]), dtype=int)+i
    y_test.append(yi_test)

train_stacked = np.vstack(train)
y_train = np.hstack(y_train)

test_stacked = np.vstack(test)
y_test = np.hstack(y_test)

In [6]:
# Reducing data dim
dim = 2

pca_dim = PCA(n_components=dim)

train_pca = pca_dim.fit_transform(train_stacked)
test_pca = pca_dim.fit_transform(test_stacked)

In [28]:
# convert to torch
class NumbersDataset(Dataset):
    def __init__(self, samples, labels):
        self.samples = torch.from_numpy(samples).to(torch.float32)
        self.labels = torch.from_numpy(labels).to(torch.long)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]

In [33]:
# Using numbersdataset to convert to torch
batch_size = 100
hidden_size = 100
num_classes = 10 # 10 image classes
learning_rate = 0.001
num_epochs = 2

train_data = NumbersDataset(train_pca, y_train)
test_data = NumbersDataset(test_pca, y_test)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [34]:
# Create network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out) # NOT USE SOFTMAX SINCE WE USE nn.CrossEntropyLoss (it is implemented in there)
        return out

model = NeuralNet(dim, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # image shape 100, 1, 28, 28 - hence we resize to 100,784
        images = images.reshape(-1, dim)
        labels = labels

        # forward
        outputs = model(images)
        #print(outputs, labels)
        loss = criterion(outputs, labels)

        # backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f"epoch: {epoch+1} / {num_epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}")

# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, dim)
        labels = labels
        outputs = model(images) # trained model

        _, pred = torch.max(outputs, 1) #value, index
        n_samples += labels.shape[0]
        n_correct += (pred == labels).sum().item()

    acc = 100.0*n_correct/n_samples
    print(f"accuracy = {acc}")

epoch: 1 / 50, step 100 / 600, loss = 6.4458
epoch: 1 / 50, step 200 / 600, loss = 2.4653
epoch: 1 / 50, step 300 / 600, loss = 2.0812
epoch: 1 / 50, step 400 / 600, loss = 3.1145
epoch: 1 / 50, step 500 / 600, loss = 2.5993
epoch: 1 / 50, step 600 / 600, loss = 2.3721
epoch: 2 / 50, step 100 / 600, loss = 3.0284
epoch: 2 / 50, step 200 / 600, loss = 2.5636
epoch: 2 / 50, step 300 / 600, loss = 2.3103
epoch: 2 / 50, step 400 / 600, loss = 1.9345
epoch: 2 / 50, step 500 / 600, loss = 2.6874
epoch: 2 / 50, step 600 / 600, loss = 2.0207
epoch: 3 / 50, step 100 / 600, loss = 2.1289
epoch: 3 / 50, step 200 / 600, loss = 2.1093
epoch: 3 / 50, step 300 / 600, loss = 2.1248
epoch: 3 / 50, step 400 / 600, loss = 2.0304
epoch: 3 / 50, step 500 / 600, loss = 3.2490
epoch: 3 / 50, step 600 / 600, loss = 2.2196
epoch: 4 / 50, step 100 / 600, loss = 2.4366
epoch: 4 / 50, step 200 / 600, loss = 2.3403
epoch: 4 / 50, step 300 / 600, loss = 3.0525
epoch: 4 / 50, step 400 / 600, loss = 2.3525
epoch: 4 /